In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=50, 
    zl_max=2.5, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 17500.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 500000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [2]:
current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    table = Table()

    fn = h5py.File(f"./samples/lenses_{count_draw}.hdf5", "r")
    src_thetaE = fn['source/thetaE'][()]
    for ii in range(src_thetaE.shape[0]):
        table[f'thetaE_s{ii}'] =  src_thetaE[ii, :]
        table[f'mass_s{ii}'] = fn['source/einstein_mass'][()][ii, :]
        table[f'z_s{ii}'] = fn['source']['z'][()][ii, :]
        table[f're_s{ii}'] = fn['source']['Re'][()][ii, :]
        table[f'q_s{ii}'] = fn['source']['q'][()][ii, :]
        table[f'pa_s{ii}'] = fn['source']['pa'][()][ii, :]
        table[f'x_s{ii}'] = fn['source']['xs'][()][ii, :]
        table[f'y_s{ii}'] = fn['source']['ys'][()][ii, :]
        table[f'mag_g_s{ii}'] = fn['source']['app_mag_g'][()][ii, :]
        table[f'mag_r_s{ii}'] = fn['source']['app_mag_r'][()][ii, :]
        table[f'mag_i_s{ii}'] = fn['source']['app_mag_i'][()][ii, :]
        table[f'mag_z_s{ii}'] = fn['source']['app_mag_z'][()][ii, :]

    table['vdisp_l'] = fn['deflector/vdisp'][()]
    table['re_l'] = fn['deflector/Re'][()]
    table['q_l']= fn['deflector/q'][()]
    table['z_l'] = fn['deflector/z'][()]
    table['mag_g_l'] = fn['deflector/app_mag_g'][()]
    table['mag_r_l'] = fn['deflector/app_mag_r'][()]
    table['mag_i_l'] = fn['deflector/app_mag_i'][()]
    table['mag_z_l'] = fn['deflector/app_mag_z'][()]

    SNR = fn['Obs/SNR'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['SNR_g'] = SNR[0,0,:]
    table['SNR_r'] = SNR[0,1,:]
    table['SNR_i'] = SNR[0,2,:]
    table['SNR_z'] = SNR[0,3,:]
    table['SNR_stack'] = SNR[0,-2,:]
    table['SNR_color'] = SNR[0,-1,:]
    SNR_griz = SNR[0,0:-2,:]
    indices = np.argmax(SNR_griz, axis=0)
    table['best_band'] = np.array(['g', 'r', 'i', 'z'])[indices] 
    
    mu = fn['Obs/magnification'][()] #shape: [nsrc, nband, n_ideal_lens]
    table['mu'] = mu[0,-2,:] #stack band magnification
    
    ring_cond = fn['Obs/ring_cond'][()] #shape: [nsrc, nband, n_ideal_lenses]
    table['if_ring'] = ring_cond[0,-2,:] #stack band ring condition of src-0

    obs_cond = fn['Obs/detect_cond'][()] #shape: [nsrc, nband, n_ideal_lenses, 3]
    
    fn.close()

    #stack band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[0, -2, :, :] #src-0, band-stack, nlens, 3-condition
    this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [nlens]
    bool_cond = (this_obs_cond==3)
    table['if_obs_stack'] = bool_cond
        
    #individual band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[:, :-2, :, :] #remove stack band
    cond = np.sum(this_obs_cond, axis=3)[0,:,:] #cond: [nband, nlens]
    bool_cond = (cond==3).astype('int') #cond: [nband, nlens]
    bool_cond = (np.sum(bool_cond, axis=0)>0) #bool_cond" [nlens]
    table['if_obs_single'] = bool_cond

    #color band
    this_obs_cond = obs_cond.astype('int')
    this_obs_cond = this_obs_cond[0, -1, :, :] #src-0, band-color, nlens, 3-condition
    this_obs_cond = np.sum(this_obs_cond, axis=1) #shape: [nlens]
    bool_cond = (this_obs_cond==3)
    table['if_obs_color'] = bool_cond
    
    n_ideal_lens = src_thetaE.shape[1]
    
    table['noise_seed'] = np.arange(n_ideal_lens, dtype='int')
    
    return n_ideal_lens, table

n_ideal_lens, table = return_detected(0, stack=True)
table[0:5]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,SNR_color,best_band,mu,if_ring,if_obs_stack,if_obs_single,if_obs_color,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,bool,int64
1.0966084016507394,220618705190.45496,2.4813,0.04755680836013041,0.48724151839551133,56.84835836540002,-0.679774694569647,-0.19032010755702378,26.8804,26.7911,27.1784,26.9373,227.31589216009394,0.48874105992050526,0.6177436725840784,0.39480433148639105,21.44529815513444,19.494334310672166,18.780101509455797,18.338227104944824,8.565924589820026,6.563163395814961,3.5898829774557774,2.789284151760811,10.657003960170975,8.001040584544414,g,2.841061144034537,False,False,False,False,0
0.9082723155014878,175970259310.74948,1.5159,0.5564402037648424,0.9412605823120466,86.1545250265512,-0.5789947049002112,-0.1931924628290287,24.0838,24.1747,24.1369,24.0414,221.33985664690238,0.9118452408175499,0.5665284149226537,0.40504501393703773,21.618946703232307,19.677164574759914,18.940674714793,18.48724607593226,12.969372833605208,8.77984747032523,7.19702764057638,5.124299069253112,16.74841983024402,11.794360040844877,g,2.928503730589229,True,False,False,False,1
0.38279059247131786,22543047250.574535,2.6924,0.6224362743158242,0.5707525533169218,2.7386936589930166,-0.09482301618729103,-0.09138855630473552,25.152,25.1834,25.2084,25.2788,130.13643389328882,0.47330489097148853,0.3819922718130264,0.3308795450335912,22.894304998147668,20.98100573472134,20.330852787096127,19.97401167526535,4.70030744048168,3.302007194171014,2.5632875403694255,1.582388529705237,5.956653167613458,4.220682699447359,g,2.0756265269352596,False,False,False,False,2
0.8391122140185563,154058251871.80637,2.2914,0.2723488843131578,0.706443158338659,30.67631333225165,0.8141141813401891,0.17524044756906007,26.6682,26.6759,26.7667,26.6563,206.73549174676776,0.6478193997706299,0.5032317557861576,0.4669746501356521,22.51634415791473,20.6185355786966,19.674547921549365,19.171136763663434,1.9046484185473997,1.4383730247518691,1.0783594647220742,0.7911558900681148,2.5614352917540764,1.7660976094905185,g,2.0565680652071827,False,False,False,False,3
0.27236039364250964,56840094747.91324,1.5451,0.2884627534645868,0.8511899612154268,16.840827570216806,-0.08538241153107988,-0.12950851772035152,27.9129,27.7307,27.1252,26.6024,189.16734876354147,0.4828159648897842,0.6043556278860815,0.9872606591801739,27.69312251992178,24.86809436080292,23.61280950239249,22.309543157012516,0.901711187623168,0.8306253246317307,1.1828423407096846,1.222855361144366,2.0670471957079264,0.8579093999171804,z,2.6892116202674115,False,False,False,False,4


In [3]:
pool = Pool(processes=64)
vars = list(zip(list(range(Ndraw)), [True]*Ndraw))
results = pool.starmap(return_detected, vars)

n_ideal_lens = [item[0] for item in results]
n_ideal_lens = sum(n_ideal_lens)

tables = [item[1] for item in results]
stacked_table = vstack(tables)

In [4]:
mask_thetaE = (stacked_table['thetaE_s0'].data > 0.074)
stacked_table = stacked_table[mask_thetaE]
stacked_table.write('lens_samples.fits', overwrite=True)
stacked_table[0]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,SNR_color,best_band,mu,if_ring,if_obs_stack,if_obs_single,if_obs_color,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,bool,bool,bool,bool,int64
1.0966084016507394,220618705190.45496,2.4813,0.04755680836013041,0.48724151839551133,56.84835836540002,-0.679774694569647,-0.19032010755702378,26.8804,26.7911,27.1784,26.9373,227.31589216009394,0.48874105992050526,0.6177436725840784,0.39480433148639105,21.44529815513444,19.494334310672166,18.780101509455797,18.338227104944824,8.565924589820026,6.563163395814961,3.5898829774557774,2.789284151760811,10.657003960170975,8.001040584544414,g,2.841061144034537,False,False,False,False,0


In [10]:
import os
os.makedirs('catalog_csv', exist_ok=True)

mask = stacked_table['if_obs_stack'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_stack.csv", overwrite=True)
print(len(table_detect), len(stacked_table), len(table_detect)/len(stacked_table))

mask = stacked_table['if_obs_single'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_single.csv", overwrite=True)
print(len(table_detect), len(stacked_table), len(table_detect)/len(stacked_table))

mask = stacked_table['if_obs_color'].data
table_detect = stacked_table[mask]
table_detect.write("catalog_csv/csst_wf_color.csv", overwrite=True)
print(len(table_detect), len(stacked_table), len(table_detect)/len(stacked_table))

161548 5186409 0.031148334039987978
76396 5186409 0.014730037681177864
54043 5186409 0.010420119200009102
